In [2]:
import torch
import torch.nn as nn
import torch.optim as optim

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

print(f'GPU available: {torch.cuda.is_available()}')

Sun May  2 18:34:18 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P0    30W / 250W |      2MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
!pip install emoji

     |████████████████████████████████| 133kB 5.1MB/s 


In [5]:
import StockSentimentDataset
from torch.utils.data import DataLoader
from sklearn.utils.class_weight import compute_class_weight

train_ds, val_ds, test_ds, embeddings, featurizer = StockSentimentDataset.load_all_data()

class_weights = compute_class_weight(class_weight='balanced', classes=[0,1,2], y=train_ds.labels)
class_weights_tensor = torch.FloatTensor(class_weights)

In [87]:
# import torch.nn.functional as f
from torch import nn

class LSTM(nn.Module):
 
  def __init__(self,  vocab_size, DIM_EMB=10, DIM_HID=10):
    super(LSTM, self).__init__()
    NUM_TAGS = 3
    (self.DIM_EMB, self.NUM_TAGS) = (DIM_EMB, NUM_TAGS)
    self.embeddings = nn.Embedding(vocab_size, DIM_EMB, padding_idx = 0)
    self.lstm = nn.LSTM(DIM_EMB, DIM_HID, bidirectional = False, batch_first = False)
    self.dropout = nn.Dropout()
    self.linear = nn.Linear(DIM_HID, NUM_TAGS)
    self.logsoftmax = nn.LogSoftmax(dim = 1)
    self.hidden = DIM_HID

  def forward(self, X, hidden, train=False):
    embedded = self.embeddings(X)
    out, hidden = self.lstm(embedded, hidden)

    lstm_out = out[-1,:,:]
    lstm_out = self.dropout(lstm_out)
   
    output = self.linear(lstm_out)
    output = self.logsoftmax(output)
    return output, hidden




In [91]:
from sklearn.metrics import f1_score
import random

def score_model(model, test_ds):
  model.eval()
  with torch.no_grad():
    test_dl = DataLoader(dataset=test_ds, batch_size=len(test_ds.samples),
      shuffle=False, collate_fn=StockSentimentDataset.collate_fn)
    predictions = []
    hidden = None
    for x, y in test_dl:
      x = x.cuda()
      y = y.cuda()
      if (hidden == None):
        hidden = (torch.zeros(1, x.shape[1], 500).cuda(), torch.zeros(1, x.shape[1], 500).cuda())
      pred_scores, hidden = lstm.forward(x, hidden)
      predictions.extend(torch.argmax(pred_scores, dim=1).tolist())

  return f1_score(test_ds.labels, predictions, labels=[0,1,2], average='micro')

In [93]:
#Training

from random import sample
import tqdm
import os
import subprocess
import random
from pretrained_embeddings import add_embeddings_to_embedding_matrix




#Training

from random import sample
import tqdm
import os
import subprocess
import random
from pretrained_embeddings import add_embeddings_to_embedding_matrix


def train(model, best_val_model, train_ds, val_ds, class_weights):

  class_weights = class_weights.cuda()
  optimizer = optim.Adam(model.parameters(), lr=0.001)
  loss_fn = torch.nn.CrossEntropyLoss()

  train_dl = DataLoader(dataset=train_ds, batch_size=len(train_ds.samples),
    shuffle=True, collate_fn=StockSentimentDataset.collate_fn)

  val_dl = DataLoader(dataset=val_ds, batch_size=len(val_ds.samples),
    shuffle=False, collate_fn=StockSentimentDataset.collate_fn)

  epoch_stats = []
  best_f1 = 0.0
  n_epochs_without_improvement = 0
  best_val_model.load_state_dict(model.state_dict())
  
  for epoch in range(200):
    hidden = None
    totalLoss = 0.0
    n_samples = 0
    model.train()
    for x, y in tqdm.notebook.tqdm(train_dl, leave=False):
      x = x.cuda()
      y = y.cuda()

      if (hidden == None):
        hidden = (torch.zeros(1, x.shape[1], 500).cuda(), torch.zeros(1, x.shape[1], 500).cuda())
      model.zero_grad()
      y = torch.nn.functional.one_hot(y, 3)
      probs, hidden = lstm.forward(x, hidden)
     
      loss = (torch.neg(probs * y)).sum()

      loss.backward()
      totalLoss += loss
      n_samples += x.size()[1]
      optimizer.step()
     


    average_train_loss = totalLoss / float(n_samples)

    model.eval()
    total_val_loss = 0.0
    n_val_samples = 0
    hidden_val = None
    with torch.no_grad():
      for x, y in val_dl:
        x = x.cuda()
        y = y.cuda()
       

        y = torch.nn.functional.one_hot(y, 3)

        if (hidden_val == None):
          hidden_val = (torch.zeros(1, x.shape[1], 500).cuda(), torch.zeros(1, x.shape[1], 500).cuda())
        probs, hidden_val = lstm.forward(x, hidden_val)
        loss = (torch.neg(probs * y)).sum()
        total_val_loss += loss
        n_val_samples += x.size()[1]
    
    average_val_loss = total_val_loss / float(n_val_samples)
    epoch_stats.append((epoch, average_train_loss, average_val_loss))
    val_f1 = score_model(model, val_ds)
    if epoch % 1 == 0:
      print("Epoch {0} finished. Train loss: {1}, Val loss: {2}".format(epoch, average_train_loss, average_val_loss))

    if val_f1 > best_f1:
      best_f1 = val_f1
      n_epochs_without_improvement = 0
      best_val_model.load_state_dict(model.state_dict())
    # else:
    #   n_epochs_without_improvement += 1
    #   if n_epochs_without_improvement == 10:
    #     break

  return val_f1, epoch_stats


lstm = LSTM(featurizer.vocab_size, DIM_HID=500, DIM_EMB=300).cuda()
best_val_model = LSTM(featurizer.vocab_size, DIM_HID=500, DIM_EMB=300)
add_embeddings_to_embedding_matrix(embeddings, lstm.embeddings, featurizer)
# model = model.cuda()
best_val_model = best_val_model.cuda()
train(lstm, best_val_model, train_ds, val_ds, class_weights_tensor)



Epoch 0 finished. Train loss: 1.0980079174041748, Val loss: 1.0920788049697876


Epoch 1 finished. Train loss: 1.0919101238250732, Val loss: 1.0888073444366455


Epoch 2 finished. Train loss: 1.0862133502960205, Val loss: 1.089893102645874


Epoch 3 finished. Train loss: 1.0829150676727295, Val loss: 1.1032720804214478


Epoch 4 finished. Train loss: 1.0841395854949951, Val loss: 1.109053134918213


...

Epoch 196 finished. Train loss: 0.5738632082939148, Val loss: 1.9527217149734497


Epoch 197 finished. Train loss: 0.5713602304458618, Val loss: 1.974109411239624


Epoch 198 finished. Train loss: 0.5732933282852173, Val loss: 1.9865504503250122


Epoch 199 finished. Train loss: 0.5737866163253784, Val loss: 1.9952433109283447


(0.44387755102040816,
 [(0,
   tensor(1.0980, device='cuda:0', grad_fn=<DivBackward0>),
   tensor(1.0921, device='cuda:0')),
  (1,
   tensor(1.0919, device='cuda:0', grad_fn=<DivBackward0>),
   tensor(1.0888, device='cuda:0')),
  (2,
   tensor(1.0862, device='cuda:0', grad_fn=<DivBackward0>),
   tensor(1.0899, device='cuda:0')),
  (3,
   tensor(1.0829, device='cuda:0', grad_fn=<DivBackward0>),
   tensor(1.1033, device='cuda:0')),
  (4,
   tensor(1.0841, device='cuda:0', grad_fn=<DivBackward0>),
   tensor(1.1091, device='cuda:0')),
  (5,
   tensor(1.0849, device='cuda:0', grad_fn=<DivBackward0>),
   tensor(1.1096, device='cuda:0')),
  (6,
   tensor(1.0832, device='cuda:0', grad_fn=<DivBackward0>),
   tensor(1.1106, device='cuda:0')),
  (7,
   tensor(1.0826, device='cuda:0', grad_fn=<DivBackward0>),
   tensor(1.1121, device='cuda:0')),
  (8,
   tensor(1.0823, device='cuda:0', grad_fn=<DivBackward0>),
   tensor(1.1141, device='cuda:0')),
  (9,
   tensor(1.0833, device='cuda:0', grad_fn=<D

In [95]:
print(score_model(lstm, train_ds))
print(score_model(lstm, test_ds))
print(score_model(lstm, val_ds))

print(score_model(best_val_model, train_ds))
print(score_model(best_val_model, test_ds))
print(score_model(best_val_model, val_ds))

0.6952695269526953
0.4256410256410256
0.44387755102040816
0.6952695269526953
0.4256410256410256
0.44387755102040816
